# Robot Control

Kevin J. Walchko

created 20 Oct

-------------------

Now we are going to start getting the Roomba to move. In order to do that, we need to program the robot correctly. Students usually are lazy and write a lazy software flow for their robot and then don't understand why things crash or don't work like they want. Once we understand the Create 2 API and program a modular/logical software flow, we will develop a simple algorithm to get our robot to do obstacle avoidance.

## Objectives

- understand how to get the Roomba to move
- understand simple obstacle avoidance
- understand a simple mobile robot software flow

## References

- [pycreate2 python software](https://pypi.python.org/pypi/pycreate2)

## Setup

In [ ]:
%matplotlib inline
from __future__ import print_function
from __future__ import division
import numpy as np

# Robot Software Architecture

There are many ways to setup how your robot's software should function. One extreme way is using the [Robot Operation System (ROS)](www.ros.org) which is not an operating system but rather an architecture for setting up a robot. On large systems, this is a good match, but there is a very steep learning curve and only runs on Ubuntu Linux.

- ROS pros
  - Free (BSD licensed) robotic architecture
  - Distributed architecture, supports both multi-process/core and multi-machine
    - Publish/subscribe architecture allows this flexibility
  - C++ and python interfaces
  - Broad academic adoption
- ROS cons
  - Rapid development cycle, leads to code working one year and broken due to new interfaces the next
    - Or you can just stay on an older version of ROS, but then you loose new advancements
  - Steep learning curve ... nothing is simple
  - Only can develop/run on Ubuntu Linux
  - Many packages written by Universities and are not always maintained to the current version or support is often lacking
  - Updated annually, but core packages do not always use the most current version for libraries
  - Complex build system with Catkin
  - Designed for relatively powerful computers and not optimized for small embedded devices like Raspberry Pi
  - No security, robots easily hacked
  - Reliant on `roscore` as the central pub/sub broker, moving large amounts of data around in messages (i.e., 3D lidar point clouds, large images, etc) can introduce unnecessary delays and CPU overhead
    - *Note:* this can be overcome by development of few nodes and smart partitioning of algorithms. However, this is not the default ROS mentality

This class will not throw you into the deep end with ROS. Instead we are just using Python.

# Simple Autonomous Behavior

Let's start off simple. Let's define a couple of states and transitions between them. Understand there is more than one way to do this, but we will give you an example

![](pics/state-diagram.png)

- **Start:** The initial state. On start-up, the robot will initialize and setup some things, then immediately tranision the Wander state.

- **Wander:** When the robot enters into this state it starts to wander around. Basically what it does is drives froward until it encounters an obstacle. It then turns and continues forward. This basically untils forever, or until the batteries run out or someone turns the robot off.
  - **Turing:** you can always turn in one direction (i.e., right turns) or alternate turns.
  - **Other:** if your robot isn't able to detect obstacles very far in-front of it, you could back up a little bit first, to give the robot some room, before the robot turns.

Let's maybe try to make it a little more robust. Now we create a couple more transitions.

- **Done:** There is some simple measureable goal we want our robot to reach. When it meets it (i.e., completely explore its environment, vacuum a room, etc), it transitions from Wander to Stop.
- **Stuck:** basically this is saying, after so many continous turns, the robot is stuck and needs human intervention. Maybe the robot has somehow become trapped in a box and can't get out. Below was a test, where an autonomous car was designed to understand common road markers and the dashed/solid line was a typical commuter lane marker. The car knew it could cross the dotted line (ignoring the solid line) to get into the circle. However, once it was inside, the only way to get out was to cross a solid white line. This was aginst the car's programming, because commuter 

<img src="pics/autonomous_trap.jpg" width="400px">

![](pics/state-diagram-2.png)

## not sure i like this
## Suggested Simple Robot Architecture for Your Roomba

Actually you can write a rather professional, modular, and clean architecture with Python. Remember to always setup your system properly and tear it down when you shutdown. Killing your software with Ctrl-C (essentially causing it to crash) is sloppy and can leave you in a bad state.

```python
#!/usr/bin/env python

from __future__ import print_function, division

STATE_START = 0
STATE_WANDER = 1
STATE_OBSTACLE_AVOID = 2
STATE_STOP  = 3

def Start(robot):
    return STATE_WANDER

def ObstacleAvoid(robot):
    next_state = STATE_WANDER
    
    sensors = robot.get_sensors()
    if 
    return next_state

def Wander(robot):
    next_state = STATE_OBSTACLE_AVOID
    
    return next_state

def Stop(robot):
    next_state = STATE_OBSTACLE_AVOID
    return next_state

StateArray = {
    STATE_START: Start,
    STATE_OBSTACLE_AVOID: ObstacleAvoid,
    STATE_WANDER: Wander,
    STATE_STOP: Stop
}
```

```python
#!/usr/bin/env python

from __future__ import print_function, division
# get states
from states import StateArray
from states import STATE_START, STATE_WANDER, STATE_OBSTACLE_AVOID, STATE_STOP
from pycreate2 import Create2            # Roomba driver
from nxp_imu import IMU                  # imu driver
from time import sleep

class MyRobot(object):
    """
    This is a super simple class to hold things together.
    
    Unfortunately you are not trained like the rest of the world to
    program with classes, so I want to keep this simple. However, if
    you have any talent, then please try to do this properly.
    """
    bot = None
    camera = None
    imu = None
    
    def __init__(self, port):
        self.bot = Create2(port)
        self.camera = Camera('pi')
        self.camera.init(window=(640,480))
        
    def __del__(self):       
        self.bot.safe()
        self.bot.close()
        self.camera.close()
        self.imu.close()

if __name__ == "__main__":
    
    port = "/dev/serial"  # serial port path, change as appropriate
    bot = MyRobot(port)
    current_state = STATE_START
    
    try:
        while True:
            current_state = StateArray[current_state](bot)
            sleep(0.1)
    except KeyboardInterrupt:
        print('User hit Ctrl-C, robot shutting down\n\nBye ...')
                
```

# Making the Create Move

So we have already talked about mobile robots, coordinate systems, body frames, etc. Now we are going to talk about how do we command the robot and get it to go where we want.

```python
from  pycreate2 import Create2

# Create a Create2.
bot = Create2()

# Start the Create 2
bot.start()

# Put the Create2 into 'safe' mode so we can drive it
# This will still provide some protection
bot.safe()

# directly set the motor speeds ... go forward
bot.drive_direct(100, 100)  # inputs for motors are +/- 500 max
time.sleep(2)

# turn in place, CW
bot.drive_direct(200,-200)
time.sleep(2)
```

In [1]:
class Ball(object):
    def __init__(self, x):
        self.x = x
    def __str__(self):
        return str(self.x)

def Test(a):
    a.x += 1

b = Ball(5)

for _ in range(3):
    print(b)
    Test(b)

5
6
7
